# Baseball Type Prediction

    Living in Houston, it would be sacreligious for me to not be a huge fan of baseball and the Hosuton Astros. For years, I've cheered, cried, and laughed as the Astros went from a pretty bad team to arguably the best team in the league (no matter what the Yankees say!). We acquired some of the best batters in the league as well as one of the greatest pitching lineups in MLB history. 

In [125]:
#importing some necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import pybaseball as pb
import numpy as np

In [126]:
raw_data = pb.statcast_pitcher('2022-04-07', '2022-10-05', player_id = 519151)
raw_data.info()

Gathering Player Data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 92 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   pitch_type                       711 non-null    object 
 1   game_date                        711 non-null    object 
 2   release_speed                    711 non-null    float64
 3   release_pos_x                    711 non-null    float64
 4   release_pos_z                    711 non-null    float64
 5   player_name                      711 non-null    object 
 6   batter                           711 non-null    int64  
 7   pitcher                          711 non-null    int64  
 8   events                           181 non-null    object 
 9   description                      711 non-null    object 
 10  spin_dir                         0 non-null      float64
 11  spin_rate_deprecated             0 non-null      float64
 12  

In [127]:
last_pitch = raw_data.get(["pitch_name"]).shift(-1)
last_pitch = last_pitch.mask(last_pitch.eq('None')).dropna()
second_last = raw_data.get(["pitch_name"]).shift(-2)
second_last = second_last.mask(second_last.eq('None')).dropna()

In [128]:
df = raw_data.get(["game_date","pitch_number", "pitch_name", "strikes", "balls", "stand"])
df.insert(3, "last_pitch", last_pitch)
df.insert(4, "second_last", second_last)

df

,game_date,pitch_number,pitch_name,last_pitch,second_last,strikes,balls,stand
0,2022-10-05,7,Changeup,4-Seam Fastball,Curveball,2,3,R
1,2022-10-05,6,4-Seam Fastball,Curveball,Slider,2,2,R
2,2022-10-05,5,Curveball,Slider,Slider,2,1,R
3,2022-10-05,4,Slider,Slider,Slider,2,1,R
4,2022-10-05,3,Slider,Slider,Curveball,1,1,R
...,...,...,...,...,...,...,...,...
706,2022-04-07,1,Curveball,4-Seam Fastball,Slider,0,0,L
707,2022-04-07,3,4-Seam Fastball,Slider,Slider,0,2,R
708,2022-04-07,2,Slider,Slider,4-Seam Fastball,0,1,R
709,2022-04-07,1,Slider,4-Seam Fastball,NaN,0,0,R


In [129]:
df = df.mask(df['pitch_number'].eq(1)).dropna()
df = df.mask(df['pitch_number'].eq(2)).dropna()

df['pitch_name'] = df['pitch_name'].replace('4-Seam Fastball', 'fast')
df['pitch_name'] = df['pitch_name'].replace(['Curveball', 'Slider'], 'breaking')


df['last_pitch'] = df['last_pitch'].replace('4-Seam Fastball', 'fast')
df['last_pitch'] = df['last_pitch'].replace(['Curveball', 'Slider'], 'breaking')

df['second_last'] = df['second_last'].replace('4-Seam Fastball', 'fast')
df['second_last'] = df['second_last'].replace(['Curveball', 'Slider'], 'breaking')

df

,game_date,pitch_number,pitch_name,last_pitch,second_last,strikes,balls,stand
0,2022-10-05,7.0,Changeup,fast,breaking,2.0,3.0,R
1,2022-10-05,6.0,fast,breaking,breaking,2.0,2.0,R
2,2022-10-05,5.0,breaking,breaking,breaking,2.0,1.0,R
3,2022-10-05,4.0,breaking,breaking,breaking,2.0,1.0,R
4,2022-10-05,3.0,breaking,breaking,breaking,1.0,1.0,R
...,...,...,...,...,...,...,...,...
699,2022-04-10,4.0,breaking,fast,fast,2.0,1.0,R
700,2022-04-10,3.0,fast,fast,fast,1.0,1.0,R
703,2022-04-07,4.0,breaking,fast,fast,2.0,1.0,L
704,2022-04-07,3.0,fast,fast,breaking,1.0,1.0,L


In [130]:
df["pitch_name"] = df["pitch_name"].astype('category')
df["pitch_name"] = df["pitch_name"].cat.codes

df["last_pitch"] = df["last_pitch"].astype('category')
df["last_pitch"] = df["last_pitch"].cat.codes

df["second_last"] = df["second_last"].astype('category')
df["second_last"] = df["second_last"].cat.codes

df['strikes'] = df['strikes'].astype(int)
df['balls'] = df['balls'].astype(int)
df['pitch_number'] = df['pitch_number'].astype(int)

df['stand'] = df['stand'].astype('category')
df['stand'] = df['stand'].cat.codes

df = df.drop('game_date', 1)

df

C:\Users\zayan\AppData\Local\Temp\ipykernel_20916\3902767926.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('game_date', 1)


,pitch_number,pitch_name,last_pitch,second_last,strikes,balls,stand
0,7,0,2,1,2,3,1
1,6,2,1,1,2,2,1
2,5,1,1,1,2,1,1
3,4,1,1,1,2,1,1
4,3,1,1,1,1,1,1
...,...,...,...,...,...,...,...
699,4,1,2,2,2,1,1
700,3,2,2,2,1,1,1
703,4,1,2,2,2,1,0
704,3,2,2,1,1,1,0


In [131]:
X = np.array(df.drop('pitch_name', 1))
y = np.array(df['pitch_name'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 101)

C:\Users\zayan\AppData\Local\Temp\ipykernel_20916\1868724527.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df.drop('pitch_name', 1))


In [132]:
clf = RandomForestClassifier(n_estimators = 200)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.6847826086956522

In [133]:
rbf = svm.SVC(kernel='rbf', gamma=10, C=1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=2, C=0.13).fit(X_train, y_train)

In [134]:
poly_pred = poly.predict(X_test)
rbf_pred = rbf.predict(X_test)

In [135]:
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (Polynomial Kernel):  71.74
F1 (Polynomial Kernel):  64.93
Accuracy (RBF Kernel):  69.57
F1 (RBF Kernel):  65.45
